# Single-machine task graph execution

The [no-setup default](https://docs.dask.org/en/latest/setup.html), according to the docs.

In [1]:
from pi_workload import define_pi_workload

In [2]:
from dask.diagnostics import ProgressBar

In [3]:
import numpy, time

The `.compute()` method is a Dask collections "build-in" and allows larger-than-memory calculations with a local scheduler on a local Dask worker process. (For details especially on the scheduling mode, see [the official docs](https://docs.dask.org/en/latest/setup/single-machine.html).)

In [4]:
with ProgressBar():
    
    start = time.time()
    pi = define_pi_workload().compute(scheduler='threads')
    elapse = time.time() - start

print('pi computed:   ', pi)
print('pi from numpy: ', numpy.pi)
print('wall time: ', elapse, 'sec')

workload in giga bytes: 250.0
500.0 chunks to process
[########################################] | 100% Completed |  1min  7.3s
pi computed:    3.141581539328
pi from numpy:  3.141592653589793
wall time:  67.41223955154419 sec


This apparently takes time, so let's watch local CPU and memory utilization with `top -u $USER`.

## What happens for too big array chunk sizes?
Let's be a bit bold and specify 8 GB instead of the default 500 MB,

In [ ]:
with ProgressBar():
    
    start = time.time()
    pi = define_pi_workload(
        chunk_size_in_megabytes=8000
    ).compute(scheduler='threads')
    elapse = time.time() - start

print('pi computed:   ', pi)
print('pi from numpy: ', numpy.pi)
print('wall time: ', elapse, 'sec')

workload in giga bytes: 250.0
31.25 chunks to process
[                                        ] | 0% Completed | 21.6s

The task graph execution fails already after several seconds here. This is because the single-machine scheduler stopped the execution as memory utilization got closer to (in this case!) 100%. In this example, this happened very quickly and actually none of the tasks got executed properly. To briefly demonstrate our very bad chunk size choice from above,

In [6]:
chosen_chunk_size = 8000 # MB

machine_mem = 125 # GiB
machine_cpus = 16
conversion_factor = 2**30 / 10**6

this_is_certainly_not_several_times = chosen_chunk_size/conversion_factor*machine_cpus/machine_mem
print(this_is_certainly_not_several_times)

0.95367431640625


## Python environment

In [7]:
!conda list --explicit

# This file may be used to create an environment using:
# $ conda create --name <env> --file <this file>
# platform: linux-64
@EXPLICIT
https://conda.anaconda.org/conda-forge/linux-64/_libgcc_mutex-0.1-conda_forge.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/ca-certificates-2020.11.8-ha878542_0.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/ld_impl_linux-64-2.35.1-hed1e6ac_0.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/libgfortran5-9.3.0-he4bcb1c_17.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/libstdcxx-ng-9.3.0-h2ae2ef3_17.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/pandoc-2.11.1.1-h36c2ea0_0.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/libgfortran-ng-9.3.0-he4bcb1c_17.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/libgomp-9.3.0-h5dbcf3e_17.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/_openmp_mutex-4.5-1_gnu.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/libgcc-ng-9.3.0-h5dbcf3e_17.tar.bz2
ht